In [0]:
print("Hi databricks")

Hi databricks


In [0]:
spark.conf.set(
    "fs.azure.account.key.gopiststorageaccount.dfs.core.windows.net","+6dUk0U2byP5OhULX9X6NPVbkTbRhDk/l8fs/RdLWR7g5SQGsVsgE4sj6+T1q7BFFHIj4GnYfs1g+AStuEjhKA=="
                 )

In [0]:
dbutils.fs.ls("abfss://marketing@gopiststorageaccount.dfs.core.windows.net/")

[FileInfo(path='abfss://marketing@gopiststorageaccount.dfs.core.windows.net/MarketingRawData_CSV.csv', name='MarketingRawData_CSV.csv', size=1453, modificationTime=1759131394000),
 FileInfo(path='abfss://marketing@gopiststorageaccount.dfs.core.windows.net/output/', name='output/', size=0, modificationTime=1759135479000)]

In [0]:
file_location = "abfss://marketing@gopiststorageaccount.dfs.core.windows.net/"

df = spark.read.format("csv").option("inferSchema",True).option("header", True).load(file_location)

# display(df)

In [0]:
from pyspark.sql.functions import when, col, to_date, regexp_replace

df = df.withColumn("date", 
                    when(col("date") == '2025/09/01', '2025-09-01')\
                   .when(col("date") == '09-02-2025', '2025-09-02')\
                   .otherwise(col("date")))  

df = df.withColumn("date", to_date("date")) 

df = df.dropna(subset=["impressions"])

In [0]:
df = df.withColumn("clicks", regexp_replace(col("clicks"), "[^0-9]", 0).cast("int"))

In [0]:
df = df.withColumn("spend_usd", regexp_replace(col("spend_usd"), "[^0-9]", 0).cast("int"))

In [0]:
df = df.withColumn("conversions", 
                    when(col("conversions") == '8   # duplicate', 8)\
                   .when(col("conversions") == 'missing', 0)\
                   .when(col("conversions") == '28   # duplicate', 0)\
                   .otherwise(col("conversions")).cast("int"))  

In [0]:
df = df.withColumn("impressions", col("impressions").cast("int"))
df = df.withColumn("leads", col("leads").cast("float"))

In [0]:
df = df.select("campaign_id","campaign_name","date", "impressions", "clicks","spend_usd", "conversions" )

In [0]:
dbutils.fs.mount(
    source = "wasbs://marketing@gopiststorageaccount.blob.core.windows.net",
    mount_point = "/mnt/marketing_output",
    extra_configs={"fs.azure.account.key.gopiststorageaccount.blob.core.windows.net":"+6dUk0U2byP5OhULX9X6NPVbkTbRhDk/l8fs/RdLWR7g5SQGsVsgE4sj6+T1q7BFFHIj4GnYfs1g+AStuEjhKA=="}

)

True

In [0]:
dbutils.fs.ls("/mnt/marketing_output")

[FileInfo(path='dbfs:/mnt/marketing_output/MarketingRawData_CSV.csv', name='MarketingRawData_CSV.csv', size=1453, modificationTime=1759131394000)]

In [0]:
dbutils.fs.mounts()

[MountInfo(mountPoint='/mnt/marketing', source='wasbs://marketing@gopiststorageaccount.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/mnt/marketing_output', source='wasbs://marketing@gopiststorageaccount.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(m

In [0]:
df.write.mode("overwrite").parquet("/mnt/marketing/output/")


In [0]:
jdbcHostname = "gopidbserver.database.windows.net"
jdbcPort = "1433"
jdbcDatabase = "gopidatabase"
jdbcUsername = "gopiadmin"
jdbcPassword = "Password@123"

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

connectionProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


In [0]:
# Replace 'column_name' with the actual column name causing the issue, or apply similar casting to all relevant columns.
df.write \
.mode("overwrite") \
  .mode("overwrite") \
  .jdbc(url=jdbcUrl, table="marketingdata", properties=connectionProperties)
